<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [564]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Import Packages

In [1]:
#import packages.
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 
np.set_printoptions(suppress=True)
from matplotlib.pyplot import figure
#From Scikit Learn
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [525]:
%pwd

'C:\\Users\\JM025575\\Predictive Models Class\\data'

In [526]:
cd /Users/JM025575/Predictive Models Class/data

C:\Users\JM025575\Predictive Models Class\data


# Load Data

In [527]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [528]:
train_df.shape

(891, 12)

In [529]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [530]:
test_df.shape

(418, 11)

In [531]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [532]:
train_df.columns.tolist()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

# Identify Target Variable and Move to Target to Column 0

In [533]:
# designate target variable name and move the target variable of 'Survived' to my first column for easier use.
targetName = 'Survived'
targetSeries = train_df[targetName]
#remove target from current location and insert in collum 0
del train_df[targetName]
train_df.insert(0, targetName, targetSeries)
#reprint dataframe and see target is in position 0
train_df.head()

,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Exploratory Data Analysis

In [534]:
train_df.describe()

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [535]:
#Survival average by passenger class
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [536]:
#Survival by sex
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [537]:
#Survival by number of parents/children aboard ship
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [538]:
train_df.dtypes

Survived         int64
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [539]:
train_df.head()

,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [540]:
train_df.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [541]:
train_df.groupby('Pclass').size()

Pclass
1    216
2    184
3    491
dtype: int64

In [542]:
train_df.groupby('Sex').size()

Sex
female    314
male      577
dtype: int64

In [543]:
train_df.groupby('Embarked').size()

Embarked
C    168
Q     77
S    644
dtype: int64

In [544]:
#Splitting out the title to use this instead of Name as a predictive feature
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [545]:
pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [546]:
#Need to reduce the number of categories for Title
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [547]:
pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Master,0,40
Miss,185,0
Mr,0,517
Mrs,126,0
Other,3,20


In [548]:
# Saving off PassengerID for later use
passengerid = test_df.PassengerId

In [549]:
#Dropping two columns that I do not believe will enhance the predictive power of my models
train_df = train_df.drop(['Ticket', 'Cabin', 'PassengerId', 'Name'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis=1)

In [550]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


# Z-Score Normalization

We will go through and run a z-score normalization for all numeric values to ensure no one feature has an undue influence on our models below.

In [551]:
#Transform numeric features to Z-Scores
train_df.Age=pd.DataFrame((train_df.Age - train_df.Age.mean())/train_df.Age.std())
train_df.Fare=pd.DataFrame((train_df.Fare - train_df.Fare.mean())/train_df.Fare.std())
test_df.Age=pd.DataFrame((test_df.Age - test_df.Age.mean())/test_df.Age.std())
test_df.Fare=pd.DataFrame((test_df.Fare - test_df.Fare.mean())/test_df.Fare.std())

# Categorical Variables to Dummies

In [552]:
from sklearn import preprocessing
le_dep = preprocessing.LabelEncoder()
#to convert into numbers
train_df.Survived = le_dep.fit_transform(train_df.Survived)

In [553]:
# perform data transformation. Creates dummies of any categorical feature and turns them in to their own column with values of 
# 0 or 1.
for col in train_df.columns[1:]:
	attName = col
	dType = train_df[col].dtype
	missing = pd.isnull(train_df[col]).any()
	uniqueCount = len(train_df[attName].value_counts(normalize=False))
	# discretize (create dummies)
	if dType == object:
		train_df = pd.concat([train_df, pd.get_dummies(train_df[col], prefix=col)], axis=1)
		del train_df[attName]

In [554]:
# perform data transformation. Creates dummies of any categorical feature and turns them in to their own column with values of 
# 0 or 1.
for col in test_df.columns[0:]:
	attName = col
	dType = test_df[col].dtype
	missing = pd.isnull(test_df[col]).any()
	uniqueCount = len(test_df[attName].value_counts(normalize=False))
	# discretize (create dummies)
	if dType == object:
		test_df = pd.concat([test_df, pd.get_dummies(test_df[col], prefix=col)], axis=1)
		del test_df[attName]

In [555]:
#Validating that there are no null values
feat_miss = train_df.columns[train_df.isnull().any()]
print(feat_miss)

Index(['Age'], dtype='object')


In [556]:
#Missing Age values, filling in with median values
train_df = train_df.fillna(train_df.median())

In [557]:
#Validating that there are no null values
feat_miss = test_df.columns[test_df.isnull().any()]
print(feat_miss)

Index(['Age', 'Fare'], dtype='object')


In [558]:
#Missing Age and Fare values, filling in with median values
test_df = test_df.fillna(test_df.median())

# Split Dataset into Train/Test 

I will create a training and test data set from train_df that will be used in the models below.  I will use a 33/67 split for the test vs train.

In [233]:
features = train_df.drop(['Survived'], axis = 1)
target = train_df["Survived"]

In [234]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features,target,test_size = .33, random_state = 0)

# Decision Tree Models

For my first models I will run Decision Trees to get an initial hold on how stable the data is and to check quickly for an overfitting of the models to the data.  Based on the results of these models I will determine what additional predictive models I will run.

## Decision Tree - Model 1

In [376]:
from sklearn import tree 
clf_dt = tree.DecisionTreeClassifier() #taking out of box options. 
#Call up the model to see the parameters you can tune (and their default setting)
print(clf_dt)
#Fit clf to the training data
clf_dt = clf_dt.fit(features_train, target_train)
#Predict clf DT model again test data
target_predicted_dt = clf_dt.predict(features_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


### Model Accuracy

In [377]:
DT_1 = accuracy_score(target_test, target_predicted_dt)
print(DT_1)
print(classification_report(target_test, target_predicted_dt))
print(confusion_matrix(target_test, target_predicted_dt))

0.7694915254237288
             precision    recall  f1-score   support

          0       0.81      0.83      0.82       184
          1       0.70      0.68      0.69       111

avg / total       0.77      0.77      0.77       295

[[152  32]
 [ 36  75]]


### Cross Validate Basic Model

In [378]:
#verify DT with Cross Validation
scores = cross_val_score(clf_dt, features_train, target_train, cv=10)  #it is sampling from each of the data sets, usually sampling 50% out of each
print("Cross Validation Score for each K",scores)
scores.mean()    

Cross Validation Score for each K [0.78688525 0.81666667 0.83333333 0.71666667 0.8        0.74576271
 0.62711864 0.79661017 0.79661017 0.79661017]


0.7716263776975085

## Decison Tree - Model 2

In [397]:
clf_dt = tree.DecisionTreeClassifier(splitter = 'random' , min_samples_split = 5, max_depth = 7) 
#Call up the model to see the parameters you can tune (and their default setting)
print(clf_dt)
#Fit clf to the training data
clf_dt = clf_dt.fit(features_train, target_train)
#Predict clf DT model again test data
target_predicted_dt = clf_dt.predict(features_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')


### Model Accuracy

In [398]:
DT_2 = accuracy_score(target_test, target_predicted_dt)
print(DT_2)
print(classification_report(target_test, target_predicted_dt))
print(confusion_matrix(target_test, target_predicted_dt))

0.8372881355932204
             precision    recall  f1-score   support

          0       0.85      0.90      0.87       184
          1       0.82      0.73      0.77       111

avg / total       0.84      0.84      0.84       295

[[166  18]
 [ 30  81]]


### Cross Validate Basic Model

In [399]:
#verify DT with Cross Validation
scores = cross_val_score(clf_dt, features_train, target_train, cv=10)  #it is sampling from each of the data sets, usually sampling 50% out of each
print("Cross Validation Score for each K",scores)
scores.mean()    

Cross Validation Score for each K [0.78688525 0.81666667 0.88333333 0.75       0.81666667 0.83050847
 0.69491525 0.84745763 0.86440678 0.83050847]


0.8121348522737797

I ran two Decision Tree models to start off with.  The first took the out of the box options and got an accuracy score of 0.769.  I cross validated this model 10 times and got a wide range of values from 0.62 to 0.83.  This shows me the out of the box option may have some overfitting.  I then ran a second model adjusting the splitter to random, the min_samples_split to 5 and the max_depth to 7.  This increased my accuracy score to 0.837 as well as gains in Precision and Recall to 0.82 and 0.73 respectively.  I again cross validated to see if the overfitting had been reduced.  I got a smaller range of values from 0.69 to 0.88 but still think the Decision Tree models are not my best option.

# Random Forest Models

Next I will run various Random Forest models to try and improve on my accuracy scores as well as find a model that is not overfit to the train data.

## Random Forest - Model 1

In [400]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
# train random forest model
rf = RandomForestClassifier()
rf.fit(features_train, target_train)
# test random forest model
target_predicted_rf = rf.predict(features_test)
RF_1 = accuracy_score(target_test, target_predicted_rf)
print(RF_1)
target_names = ["No", "Yes"]
print(classification_report(target_test, target_predicted_rf, target_names=target_names))

0.8169491525423729
             precision    recall  f1-score   support

         No       0.84      0.88      0.86       184
        Yes       0.78      0.72      0.75       111

avg / total       0.82      0.82      0.82       295



## Cross Validate Random Forest

In [401]:
#verify Random Forest with Cross Validation
scores_rf = cross_val_score(rf, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_rf)
scores_rf.mean()  

Cross Validation Score for each K [0.78688525 0.8        0.83333333 0.8        0.83333333 0.76271186
 0.6779661  0.84745763 0.81355932 0.83050847]


0.7985755302398815

## Random Forest - Model 2

In [407]:
rf = RandomForestClassifier(n_estimators= 900, n_jobs=4,oob_score=True)
rf.fit(features_train, target_train)
# test random forest model
target_predicted_rf = rf.predict(features_test)
RF_2 = accuracy_score(target_test, target_predicted_rf)
print(RF_2)
target_names = ["No", "Yes"]
print(classification_report(target_test, target_predicted_rf, target_names=target_names))

0.8067796610169492
             precision    recall  f1-score   support

         No       0.84      0.85      0.85       184
        Yes       0.75      0.74      0.74       111

avg / total       0.81      0.81      0.81       295



## Cross Validate Random Forest

In [408]:
#verify Random Forest with Cross Validation
scores_rf = cross_val_score(rf, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_rf)
scores_rf.mean()  

Cross Validation Score for each K [0.7704918  0.76666667 0.86666667 0.75       0.85       0.83050847
 0.69491525 0.86440678 0.83050847 0.84745763]


0.8071621746781513

## Random Forest - Model 3

In [409]:
rf = RandomForestClassifier(n_estimators= 100, criterion = 'entropy', min_samples_split = 5 )
rf.fit(features_train, target_train)
# test random forest model
target_predicted_rf = rf.predict(features_test)
RF_3 = accuracy_score(target_test, target_predicted_rf)
print(RF_3)
target_names = ["No", "Yes"]
print(classification_report(target_test, target_predicted_rf, target_names=target_names))

0.8271186440677966
             precision    recall  f1-score   support

         No       0.86      0.86      0.86       184
        Yes       0.77      0.77      0.77       111

avg / total       0.83      0.83      0.83       295



## Cross Validate Random Forest

In [410]:
#verify Random Forest with Cross Validation
scores_rf = cross_val_score(rf, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_rf)
scores_rf.mean()  

Cross Validation Score for each K [0.83606557 0.81666667 0.85       0.76666667 0.85       0.81355932
 0.71186441 0.84745763 0.86440678 0.83050847]


0.8187195517273317

I ran three Random Forest models.  The first model took the out of the box options and got an accuracy score of 0.817.  This model was cross validated 10 times and again we got too wide of a range of accuracy score from 0.67 to 0.84.  For my second Random Forest model, I adjust the n_estimators to 900, n_jobs to 4, and oob_score to True and got a lower accuracy score of 0.807.  Cross validating 10 times again showed a wider range of values ranging nearly 15%. For my final model, I set n_estimators to 100, criterion to 'entropy', and min_samples_split to 5 and increased my accuracy score to 0.827.  Cross validation showed a tighter range of accuracy score values showing that thus far my third Random Forest model is my best.

# Gradient Boosting Models

I am continuing to pursue a higher accuracy score while still not creating an overfit model.  I will run various Gradient Boosting models in my attempt to do so.

## Gradient Boosting Model 1

In [411]:
from sklearn.ensemble import GradientBoostingClassifier
clf_GBC = GradientBoostingClassifier()
clf_GBC.fit(features_train, target_train)
predicted_GBC=clf_GBC.predict(features_test)
expected = target_test
GB_1 = accuracy_score(expected,predicted_GBC)
print(GB_1)
print(classification_report(expected, predicted_GBC,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_GBC))


0.8338983050847457
             precision    recall  f1-score   support

         No       0.85      0.89      0.87       184
        Yes       0.80      0.74      0.77       111

avg / total       0.83      0.83      0.83       295

[[164  20]
 [ 29  82]]


## Cross Validate Gradient Boosting

In [412]:
#verify Gradient Boosting with Cross Validation
scores_gb = cross_val_score(clf_GBC, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_gb)
scores_gb.mean()  

Cross Validation Score for each K [0.86885246 0.81666667 0.9        0.78333333 0.85       0.81355932
 0.74576271 0.93220339 0.89830508 0.89830508]


0.8506988052236732

## Gradient Boosting Model 2

In [510]:
clf_GBC = GradientBoostingClassifier(n_estimators=200)
clf_GBC.fit(features_train, target_train)
predicted_GBC=clf_GBC.predict(features_test)
expected = target_test
GB_2 = accuracy_score(expected,predicted_GBC)
print(GB_2)
print(classification_report(expected, predicted_GBC,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_GBC))

0.8305084745762712
             precision    recall  f1-score   support

         No       0.85      0.89      0.87       184
        Yes       0.80      0.74      0.77       111

avg / total       0.83      0.83      0.83       295

[[163  21]
 [ 29  82]]


## Cross Validate Gradient Boosting

In [414]:
#verify Gradient Boosting with Cross Validation
scores_gb = cross_val_score(clf_GBC, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_gb)
scores_gb.mean()  

Cross Validation Score for each K [0.83606557 0.83333333 0.91666667 0.76666667 0.85       0.79661017
 0.77966102 0.89830508 0.84745763 0.86440678]


0.8389172918403262

## Gradient Boosting Model 3

In [511]:
clf_GBC = GradientBoostingClassifier(n_estimators=100, min_samples_split = 5)
clf_GBC.fit(features_train, target_train)
predicted_GBC=clf_GBC.predict(features_test)
expected = target_test
GB_3 = accuracy_score(expected,predicted_GBC)
print(GB_3)
print(classification_report(expected, predicted_GBC,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_GBC))

0.8372881355932204
             precision    recall  f1-score   support

         No       0.85      0.89      0.87       184
        Yes       0.81      0.75      0.78       111

avg / total       0.84      0.84      0.84       295

[[164  20]
 [ 28  83]]


## Cross Validate Gradient Boosting

In [436]:
#verify Gradient Boosting with Cross Validation
scores_gb = cross_val_score(clf_GBC, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_gb)
scores_gb.mean()  

Cross Validation Score for each K [0.83606557 0.81666667 0.9        0.76666667 0.86666667 0.81355932
 0.76271186 0.91525424 0.89830508 0.88135593]


0.8457252014448459

I ran three Gradient Boosting models.  The first model took the out of the box options and got an accuracy score of 0.834.  I cross validated 10 times and saw better results than in previous models with a roughly 15% range in accuracy values.  This is still too high but we are improving.  My second model I adjusted n_estimators to 200 and got an accuracy score of 0.830 again.  Cross validation again showed similar results.  I ran one final Gradient Boosting model where I put n_estimators back to 100 and set my min_samples_split to 5 and got an accuracy score of 0.837.  This model also produced my best Precision and Recall at 0.81 and 0.75 respectively.  Cross validating 10 times still shows a wide range of roughly 15%.  

# Bagging Classifier Models

Next I will run various Bagging Classifier models with the intention of improving accuracy while creating a model that is not overfit to the data.

## Bagging Classifier Model 1

In [437]:
#Bagging Classifer
from sklearn.ensemble import BaggingClassifier
clf_bag = BaggingClassifier()
print(clf_bag)
clf_bag.fit(features_train, target_train)
predicted_bag=clf_bag.predict(features_test)
expected = target_test
BC_1 = accuracy_score(expected,predicted_bag)
print(BC_1)
print(classification_report(expected, predicted_bag,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_bag))

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)
0.8169491525423729
             precision    recall  f1-score   support

         No       0.83      0.89      0.86       184
        Yes       0.79      0.69      0.74       111

avg / total       0.82      0.82      0.81       295

[[164  20]
 [ 34  77]]


## Cross Validate Bagging Classifier

In [438]:
#verify Bagging Classifier with Cross Validation
scores_bc = cross_val_score(clf_bag, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_bc)
scores_bc.mean()  

Cross Validation Score for each K [0.80327869 0.8        0.9        0.78333333 0.83333333 0.81355932
 0.6779661  0.86440678 0.79661017 0.76271186]


0.8035199592479392

## Bagging Classifier Model 2

In [453]:
clf_bag = BaggingClassifier(n_estimators=333, random_state=21,bootstrap_features = True)
print(clf_bag)
clf_bag.fit(features_train, target_train)
predicted_bag=clf_bag.predict(features_test)
expected = target_test
BC_2 = accuracy_score(expected,predicted_bag)
print(BC_2)
print(classification_report(expected, predicted_bag,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_bag))

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=True, max_features=1.0, max_samples=1.0,
         n_estimators=333, n_jobs=1, oob_score=False, random_state=21,
         verbose=0, warm_start=False)
0.8440677966101695
             precision    recall  f1-score   support

         No       0.86      0.89      0.88       184
        Yes       0.81      0.77      0.79       111

avg / total       0.84      0.84      0.84       295

[[164  20]
 [ 26  85]]


## Cross Validate Bagging Classifier

In [448]:
#verify Bagging Classifier with Cross Validation
scores_bc = cross_val_score(clf_bag, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_bc)
scores_bc.mean()  

Cross Validation Score for each K [0.80327869 0.81666667 0.91666667 0.78333333 0.86666667 0.83050847
 0.69491525 0.88135593 0.83050847 0.83050847]


0.8254408632027415

## Bagging Classifier Model 3

In [454]:
clf_bag = BaggingClassifier(n_estimators=201, random_state = 0, n_jobs = 4)
print(clf_bag)
clf_bag.fit(features_train, target_train)
predicted_bag=clf_bag.predict(features_test)
expected = target_test
BC_3 = accuracy_score(expected,predicted_bag)
print(BC_3)
print(classification_report(expected, predicted_bag,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_bag))

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=201, n_jobs=4, oob_score=False, random_state=0,
         verbose=0, warm_start=False)
0.8169491525423729
             precision    recall  f1-score   support

         No       0.85      0.85      0.85       184
        Yes       0.76      0.76      0.76       111

avg / total       0.82      0.82      0.82       295

[[157  27]
 [ 27  84]]


## Cross Validate Bagging Classifier

In [450]:
#verify Random Forest with Cross Validation
scores_bc = cross_val_score(clf_bag, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_bc)
scores_bc.mean()  

Cross Validation Score for each K [0.78688525 0.8        0.83333333 0.78333333 0.85       0.77966102
 0.66101695 0.84745763 0.84745763 0.81355932]


0.8002704454941186

I ran 3 differnt Bagging Classifier models.  The first model took the out of the box features and got an accuracy score of 0.817.  Cross validation showed a large amount of overfitting as the accuracy scores ranged nearly 23%.  My second model I changed the n_estimators to 333, random_state to 21, and bootstrap_features to True and increased my accuracy score to 0.844, my highest thus far.  Cross validation again though showed too wide of a range of results.  My final Bagging Classifier model I set the n_estimators to 201, random_state to 0 and n_jobs to 4. This resulted in an accuracy score of 0.817.  Cross validation again confirmed that the Bagging Classifier models are too overfit to the data.

# KNN Models

Next I will run multiple KNN models as I continue to pursue an accurate model that is not overfit to the data.

## KNN Model 1

In [455]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
neigh = KNeighborsClassifier()
neigh.fit(features_train, target_train) 
predicted_KNN1 = neigh.predict(features_test)
print(neigh)
# make predictions
#print(target_test)
# summarize the fit of the model
print(classification_report(target_test, predicted_KNN1))
print(confusion_matrix(target_test, predicted_KNN1))
KNN_1 = accuracy_score(target_test,predicted_KNN1)
print(KNN_1)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

          0       0.86      0.85      0.85       184
          1       0.75      0.77      0.76       111

avg / total       0.82      0.82      0.82       295

[[156  28]
 [ 26  85]]
0.8169491525423729


## Cross Validate KNN

In [456]:
#verify KNN with Cross Validation
scores_KNN = cross_val_score(neigh, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_KNN)
scores_KNN.mean()  

Cross Validation Score for each K [0.81967213 0.78333333 0.9        0.76666667 0.83333333 0.84745763
 0.74576271 0.86440678 0.81355932 0.84745763]


0.8221649532277484

## KNN Model 2

In [457]:
neigh = KNeighborsClassifier(n_neighbors=7, p = 1)
neigh.fit(features_train, target_train) 
predicted_KNN1 = neigh.predict(features_test)
print(neigh)
# make predictions
#print(target_test)
# summarize the fit of the model
print(classification_report(target_test, predicted_KNN1))
print(confusion_matrix(target_test, predicted_KNN1))
KNN_2 = accuracy_score(target_test,predicted_KNN1)
print(KNN_2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=1,
           weights='uniform')
             precision    recall  f1-score   support

          0       0.85      0.85      0.85       184
          1       0.75      0.75      0.75       111

avg / total       0.81      0.81      0.81       295

[[156  28]
 [ 28  83]]
0.8101694915254237


## Cross Validate KNN

In [458]:
#verify KNN with Cross Validation
scores_KNN = cross_val_score(neigh, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_KNN)
scores_KNN.mean()  

Cross Validation Score for each K [0.86885246 0.81666667 0.86666667 0.75       0.8        0.77966102
 0.76271186 0.88135593 0.88135593 0.88135593]


0.828862647031583

## KNN Model 3

In [472]:
neigh = KNeighborsClassifier(n_neighbors = 13, p = 1, weights = 'distance')
neigh.fit(features_train, target_train) 
predicted_KNN1 = neigh.predict(features_test)
print(neigh)
# make predictions
#print(target_test)
# summarize the fit of the model
print(classification_report(target_test, predicted_KNN1))
print(confusion_matrix(target_test, predicted_KNN1))
KNN_3 = accuracy_score(target_test,predicted_KNN1)
print(KNN_3)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=13, p=1,
           weights='distance')
             precision    recall  f1-score   support

          0       0.84      0.86      0.85       184
          1       0.75      0.72      0.74       111

avg / total       0.81      0.81      0.81       295

[[158  26]
 [ 31  80]]
0.8067796610169492


## Cross Validate KNN

In [473]:
#verify KNN with Cross Validation
scores_KNN = cross_val_score(neigh, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_KNN)
scores_KNN.mean()  

Cross Validation Score for each K [0.80327869 0.78333333 0.86666667 0.78333333 0.83333333 0.83050847
 0.71186441 0.88135593 0.81355932 0.88135593]


0.8188589422987868

I ran three KNN models.  The first model used the out of the box parameters and got an accuracy score of 0.817. I cross validated to check for model overfitting, and similar to previous models see signs of this happening with accuracy scores ranging from 0.74 to 0.9.  My second KNN model I set n_neighbors to 7 and p = 1 and got an accuracy score of 0.810.  Cross validation did show less signs of overfitting with the range this time between 0.75 and 0.88.  My final KNN model had n_neighbors set to 13, p = 1, and weights = 'distance'.  This produced an accuracy score of 0.807.  I cross validated this model as well to similar results with too wide a range of values

# SVM RBF Models

Next I will run various SVM RBF models to tru and obtain my highest accuracy score while not overfitting the model to my data.

## SVM RBF Model 1

In [474]:
#standard RBF SVM Model
clf_rbf = SVC(kernel='rbf', C=1.0, gamma='auto')
clf_rbf.fit(features_train, target_train)
predicted_SVM=clf_rbf.predict(features_test)
expected = target_test
# summarize the fit of the model
print(classification_report(expected, predicted_SVM,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_SVM))
RBF_1 = accuracy_score(expected,predicted_SVM)
print(RBF_1)

             precision    recall  f1-score   support

         No       0.85      0.87      0.86       184
        Yes       0.77      0.74      0.76       111

avg / total       0.82      0.82      0.82       295

[[160  24]
 [ 29  82]]
0.8203389830508474


## Cross Validate SVM RBF Model

In [475]:
scores_SVMR = cross_val_score(clf_rbf, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_SVMR)
scores_SVMR.mean()  

Cross Validation Score for each K [0.81967213 0.78333333 0.91666667 0.76666667 0.83333333 0.84745763
 0.77966102 0.88135593 0.88135593 0.88135593]


0.8390858571825508

## SMV RBF Model 2

In [485]:
clf_rbf = SVC(kernel='rbf', C=21.0)
clf_rbf.fit(features_train, target_train)
predicted_SVM=clf_rbf.predict(features_test)
expected = target_test
# summarize the fit of the model
print(classification_report(expected, predicted_SVM,target_names=['No', 'Yes']))
print(confusion_matrix(expected, predicted_SVM))
RBF_2 = accuracy_score(expected,predicted_SVM)
print(RBF_2)

             precision    recall  f1-score   support

         No       0.83      0.92      0.88       184
        Yes       0.85      0.69      0.76       111

avg / total       0.84      0.84      0.83       295

[[170  14]
 [ 34  77]]
0.8372881355932204


## Cross Validate SVM RBF Model

In [486]:
scores_SVMR = cross_val_score(clf_rbf, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_SVMR)
scores_SVMR.mean()  

Cross Validation Score for each K [0.81967213 0.8        0.93333333 0.78333333 0.85       0.77966102
 0.72881356 0.88135593 0.86440678 0.81355932]


0.82541354079837

I ran two separate Support Vector Machine RBF models.  The first one took the out of the box parameters and got an accuracy score of 0.820.  Cross validating 10 times showed a fair amount of overfitting with my accuracy scores ranging from 0.78 to 0.91, although still better than most of my models.  My second SVM RBF model I adjusted C to 21 and kept all other parameters the same and got an accuraracy of 0.837 and an increased Precision to 0.85. Cross validation showed similar overfitting to the first model.

# ANN Models

The final set of models I will run will be Artificial Neural Networks.  I will run 3 models in hopes of obtaining my best accuracy score and best fit model.

## ANN Model 1

In [499]:
from sklearn.neural_network import MLPClassifier
clf_ann =MLPClassifier()
clf_ann.fit(features_train, target_train)
target_predicted_ann = clf_ann.predict(features_test)
ANN_1 = accuracy_score(target_test, target_predicted_ann)
print('Accuracy', ANN_1)
target_names = ["Salaray <= 50k", "Salary >= 50k"]
print(classification_report(target_test, target_predicted_ann, target_names=target_names))
print(confusion_matrix(target_test, target_predicted_ann))

Accuracy 0.8305084745762712
                precision    recall  f1-score   support

Salaray <= 50k       0.85      0.89      0.87       184
 Salary >= 50k       0.80      0.73      0.76       111

   avg / total       0.83      0.83      0.83       295

[[164  20]
 [ 30  81]]


## Cross Validate ANN Model

In [500]:
scores_ANN = cross_val_score(clf_ann, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_ANN)
scores_ANN.mean()  

Cross Validation Score for each K [0.80327869 0.76666667 0.91666667 0.8        0.85       0.83050847
 0.72881356 0.89830508 0.86440678 0.88135593]


0.8340001852366399

## ANN Model 2

In [501]:
clf_ann =MLPClassifier(hidden_layer_sizes=(50), max_iter=500, alpha=1e-05,
                     random_state=21,tol=0.000000001)
clf_ann.fit(features_train, target_train)
target_predicted_ann = clf_ann.predict(features_test)
ANN_2 = accuracy_score(target_test, target_predicted_ann)
print('Accuracy', ANN_2)
target_names = ["Salaray <= 50k", "Salary >= 50k"]
print(classification_report(target_test, target_predicted_ann, target_names=target_names))
print(confusion_matrix(target_test, target_predicted_ann))

Accuracy 0.8271186440677966
                precision    recall  f1-score   support

Salaray <= 50k       0.83      0.91      0.87       184
 Salary >= 50k       0.82      0.69      0.75       111

   avg / total       0.83      0.83      0.82       295

[[167  17]
 [ 34  77]]


## Cross Validate ANN Model

In [502]:
scores_ANN = cross_val_score(clf_ann, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_ANN)
scores_ANN.mean()  

Cross Validation Score for each K [0.75409836 0.75       0.9        0.78333333 0.81666667 0.81355932
 0.72881356 0.89830508 0.86440678 0.83050847]


0.8139691580994721

## ANN Model 3

In [507]:
clf_ann =MLPClassifier(hidden_layer_sizes=(150,150), max_iter=500,
                     random_state=21)
clf_ann.fit(features_train, target_train)
target_predicted_ann = clf_ann.predict(features_test)
ANN_3 = accuracy_score(target_test, target_predicted_ann)
print('Accuracy', ANN_3)
target_names = ["Salaray <= 50k", "Salary >= 50k"]
print(classification_report(target_test, target_predicted_ann, target_names=target_names))
print(confusion_matrix(target_test, target_predicted_ann))

Accuracy 0.823728813559322
                precision    recall  f1-score   support

Salaray <= 50k       0.84      0.89      0.86       184
 Salary >= 50k       0.79      0.72      0.75       111

   avg / total       0.82      0.82      0.82       295

[[163  21]
 [ 31  80]]


## Cross Validate ANN Model

In [508]:
scores_ANN = cross_val_score(clf_ann, features_train, target_train, cv=10)  
print("Cross Validation Score for each K",scores_ANN)
scores_ANN.mean()  

Cross Validation Score for each K [0.7704918  0.76666667 0.86666667 0.78333333 0.83333333 0.74576271
 0.71186441 0.86440678 0.84745763 0.84745763]


0.803744095582106

I ran 3 separate neural network models.  The first model took the out of the box parameters and got an accuracy score of 0.831.  I cross validated this model 10 times and got a range of accuracy score values of nearly 20%.  I adjusted my second model to have my hidden layer size at 50, max_iter = 500, alpha = 1e-05, random_state = 21, and tol = 0.000000001.  This gave me an accuracy of 0.827.  Cross validation still showed a range of accuracy values of about 18%.  My final ANN model I simplified a bit by having all default parameters except for hidden layer sizes set to 150,150, max_iter = 500, and random_state = 21.  This produced an accuracy of 0.824.  This showed the least amount of overfitting in my cross validation with ranges from 0.71 to 0.86.

# Model Summaries

Below I will summarize the accuracy score values of all models I ran prior to choosing one that will be used with the test_df data set.

In [509]:
models = pd.DataFrame({
    'Model': ['Decision Tree 1', 'Decision Tree 2',
              'Random Forest 1', 'Random Forest 2', 'Random Forest 3',
              'Gradient Boosting 1', 'Gradient Boosting 2', 'Gradient Boosting 3',
              'Bagging Classifier 1', 'Bagging Classifier 2', 'Bagging Classifier 3', 
              'KNN 1', 'KNN 2', 'KNN 3', 
              'RBF 1', 'RBF 2', 
              'ANN 1','ANN 2','ANN 3'],
    'Score': [DT_1, DT_2, RF_1, RF_2, RF_3, GB_1, GB_2, GB_3, BC_1, BC_2, BC_3, 
                       KNN_1, KNN_2, KNN_3, RBF_1, RBF_2, ANN_1, ANN_2, ANN_3]})
models.sort_values(by='Score', ascending=False)

,Model,Score
9,Bagging Classifier 2,0.844068
1,Decision Tree 2,0.837288
7,Gradient Boosting 3,0.837288
15,RBF 2,0.837288
5,Gradient Boosting 1,0.833898
6,Gradient Boosting 2,0.833898
16,ANN 1,0.830508
17,ANN 2,0.827119
4,Random Forest 3,0.827119
18,ANN 3,0.823729


The table shows that the Bagging Classifier 2 model had my best accuracy score.  This model though showed a great amount of overfitting as did the Decision Tree 2 and the RBF 2.  Because of this, and because of the strong Precision (0.81) and Recall (0.75) as well as the relatively low level of overfitting compared to my other models, I am choosing to use the Gradient Boosting 3 model as my submission for the Kaggle competition.

# Test Predictions

In [559]:
test_df.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other
0,3,0.298099,0,0,-0.497213,0,1,0,1,0,0,0,1,0,0
1,3,1.179547,1,0,-0.512045,1,0,0,0,1,0,0,0,1,0
2,2,2.237285,0,0,-0.463974,0,1,0,1,0,0,0,1,0,0
3,3,-0.230769,0,0,-0.482308,0,1,0,0,1,0,0,1,0,0
4,3,-0.583349,1,1,-0.417469,1,0,0,0,1,0,0,0,1,0


In [560]:
test_prediction = clf_GBC.predict(test_df)
submission = pd.DataFrame({
        "PassengerId": passengerid,
        "Survived": test_prediction
    })

submission.PassengerId = submission.PassengerId.astype(int)
submission.Survived = submission.Survived.astype(int)

submission.to_csv("titanic1_submission.csv", index=False)